In [1]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data = pd.read_csv("/Users/triptibhardwaj/Downloads/zomato.csv")
data.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [3]:
print(data.isnull().sum())

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7775
votes                              0
phone                           1208
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

In [5]:
data.describe()

,votes
count,51717.000000
mean,283.697527
std,803.838853
min,0.000000
25%,7.000000
50%,41.000000
75%,198.000000
max,16832.000000


In [6]:
data=data.drop(['url','dish_liked','phone'],axis=1)

In [7]:
data.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,Banashankari,Quick Bites,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [8]:
data.duplicated().sum()
data.drop_duplicates(inplace=True)
data.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,Banashankari,Quick Bites,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [9]:
data.isnull().sum()

address                           0
name                              0
online_order                      0
book_table                        0
rate                           7767
votes                             0
location                         21
rest_type                       227
cuisines                         45
approx_cost(for two people)     345
reviews_list                      0
menu_item                         0
listed_in(type)                   0
listed_in(city)                   0
dtype: int64

In [10]:
data.dropna(how='any',inplace=True)
data.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,Banashankari,Quick Bites,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [11]:
data.isnull().sum()

address                        0
name                           0
online_order                   0
book_table                     0
rate                           0
votes                          0
location                       0
rest_type                      0
cuisines                       0
approx_cost(for two people)    0
reviews_list                   0
menu_item                      0
listed_in(type)                0
listed_in(city)                0
dtype: int64

In [12]:
data = data.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type', 'listed_in(city)':'city'})

In [13]:
data['cost'] = data['cost'].astype(str) 

In [14]:
data['cost'] = data['cost'].str.replace(',', '').astype(float)

In [15]:
data = data.loc[data.rate !='NEW']
data = data.loc[data.rate !='-'].reset_index(drop=True)
remove_slash = lambda x: x.replace('/5', '')
data.rate = data.rate.apply(remove_slash).str.strip().astype('float')

In [16]:
data.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [17]:
data.name = data.name.apply(lambda x:x.title())
data.online_order.replace(('Yes','No'),(True, False),inplace=True)
data.book_table.replace(('Yes','No'),(True, False),inplace=True)

In [18]:
data.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [19]:
restaurants = list(data['name'].unique())
data['Mean Rating'] = 0

for i in range(len(restaurants)):
    data['Mean Rating'][data['name'] == restaurants[i]] = data['rate'][data['name'] == restaurants[i]].mean()

In [20]:
data.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.118182
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,4.100000
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.800000
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.700000
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.800000


In [21]:
data.loc[data.name =='Jalsa']

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.118182
400,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Delivery,Banashankari,4.118182
485,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Dine-out,Banashankari,4.118182
1942,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Basavanagudi,4.118182
2385,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Delivery,Basavanagudi,4.118182
2775,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,804,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n Super ambience\nGreat...",[],Dine-out,Basavanagudi,4.118182
6104,"25/7, Doddannakundi, Outer Ring Road, Mahadeva...",Jalsa,True,True,4.2,2173,Marathahalli,Casual Dining,"North Indian, Mughlai",1500.0,"[('Rated 4.0', 'RATED\n A pretty decent place...",[],Delivery,Brookefield,4.118182
15334,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,783,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Jayanagar,4.118182
16242,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,783,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Delivery,Jayanagar,4.118182
16883,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,783,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Dine-out,Jayanagar,4.118182


In [22]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (1,5))
#X_scaled = ((X - X.min()) / (X.max() - X.min())) * (b - a) + a {Here, [a,b]=[1,5]}
data[['Mean Rating']] = scaler.fit_transform(data[['Mean Rating']]).round(2)

In [23]:
data.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,3.99
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,3.97
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.58
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.45
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.58


Some of the common text preprocessing / cleaning steps are:

Lower casing,
Removal of Punctuations,
Removal of Stopwords and
Removal of URLs

In [24]:
#Before Text Processing
data[['reviews_list', 'cuisines']].head()

,reviews_list,cuisines
0,"[('Rated 4.0', 'RATED\n A beautiful place to ...","North Indian, Mughlai, Chinese"
1,"[('Rated 4.0', 'RATED\n Had been here for din...","Chinese, North Indian, Thai"
2,"[('Rated 3.0', ""RATED\n Ambience is not that ...","Cafe, Mexican, Italian"
3,"[('Rated 4.0', ""RATED\n Great food and proper...","South Indian, North Indian"
4,"[('Rated 4.0', 'RATED\n Very good restaurant ...","North Indian, Rajasthani"


In [25]:
#Lower Casing
data["reviews_list"] = data["reviews_list"].str.lower()
data[['reviews_list', 'cuisines']].head()

,reviews_list,cuisines
0,"[('rated 4.0', 'rated\n a beautiful place to ...","North Indian, Mughlai, Chinese"
1,"[('rated 4.0', 'rated\n had been here for din...","Chinese, North Indian, Thai"
2,"[('rated 3.0', ""rated\n ambience is not that ...","Cafe, Mexican, Italian"
3,"[('rated 4.0', ""rated\n great food and proper...","South Indian, North Indian"
4,"[('rated 4.0', 'rated\n very good restaurant ...","North Indian, Rajasthani"


In [26]:
#Removal of Punctuations 
import string
punc = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', punc))

data["reviews_list"] = data["reviews_list"].apply(lambda text: remove_punctuation(text))
data[['reviews_list', 'cuisines']].head()

,reviews_list,cuisines
0,rated 40 ratedn a beautiful place to dine int...,"North Indian, Mughlai, Chinese"
1,rated 40 ratedn had been here for dinner with...,"Chinese, North Indian, Thai"
2,rated 30 ratedn ambience is not that good eno...,"Cafe, Mexican, Italian"
3,rated 40 ratedn great food and proper karnata...,"South Indian, North Indian"
4,rated 40 ratedn very good restaurant in neigh...,"North Indian, Rajasthani"


In [27]:
#Removal of Stopwords
Stopwords = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in Stopwords])

data["reviews_list"] = data["reviews_list"].apply(lambda text: remove_stopwords(text))
data[['reviews_list', 'cuisines']].head()

,reviews_list,cuisines
0,rated 40 ratedn beautiful place dine inthe int...,"North Indian, Mughlai, Chinese"
1,rated 40 ratedn dinner family turned good choo...,"Chinese, North Indian, Thai"
2,rated 30 ratedn ambience good enough pocket fr...,"Cafe, Mexican, Italian"
3,rated 40 ratedn great food proper karnataka st...,"South Indian, North Indian"
4,rated 40 ratedn good restaurant neighbourhood ...,"North Indian, Rajasthani"


In [28]:
#Removal of URLS
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

data["reviews_list"] = data["reviews_list"].apply(lambda text: remove_urls(text))
data[['reviews_list', 'cuisines']].head()

,reviews_list,cuisines
0,rated 40 ratedn beautiful place dine inthe int...,"North Indian, Mughlai, Chinese"
1,rated 40 ratedn dinner family turned good choo...,"Chinese, North Indian, Thai"
2,rated 30 ratedn ambience good enough pocket fr...,"Cafe, Mexican, Italian"
3,rated 40 ratedn great food proper karnataka st...,"South Indian, North Indian"
4,rated 40 ratedn good restaurant neighbourhood ...,"North Indian, Rajasthani"


In [30]:
data.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,rated 40 ratedn beautiful place dine inthe int...,[],Buffet,Banashankari,3.99
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,rated 40 ratedn dinner family turned good choo...,[],Buffet,Banashankari,3.97
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,rated 30 ratedn ambience good enough pocket fr...,[],Buffet,Banashankari,3.58
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,rated 40 ratedn great food proper karnataka st...,[],Buffet,Banashankari,3.45
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,rated 40 ratedn good restaurant neighbourhood ...,[],Buffet,Banashankari,3.58


In [31]:
data=data.drop(['address','rest_type', 'type', 'menu_item', 'votes'],axis=1)

# Recommendation by Reviews

In [32]:
df_percent = data.sample(frac=0.5)
df_percent.set_index('name', inplace=True)
indices = pd.Series(df_percent.index)

In [33]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [34]:
def recommend_like(name, top_n=-1, cosine_similarities=cosine_similarities):
    recommend_restaurant = []
    try:
        idx = indices[indices == name].index[0]
        score_series = pd.Series(cosine_similarities[idx])
        top_indexes = list(score_series.iloc[1:-1].index)

        for each in top_indexes:
            recommend_restaurant.append(list(df_percent.index)[each])

        df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])

        for each in recommend_restaurant:
            selected_data = df_percent[['cuisines', 'Mean Rating', 'cost']][df_percent.index == each].sample()
            df_new = pd.concat([df_new, selected_data])

        df_new = df_new.drop_duplicates(subset=['cuisines', 'Mean Rating', 'cost'], keep=False)
        df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(top_n)

        print(f'TOP RESTAURANTS LIKE {name.upper()}: ')
        df_new.reset_index(drop=True, inplace=True)
        return df_new
    except:
        print('No restaurants with similar reviews.')


In [35]:
data.loc[(data.name == 'Jalsa')][:1]

,name,online_order,book_table,rate,location,cuisines,cost,reviews_list,city,Mean Rating
0,Jalsa,True,True,4.1,Banashankari,"North Indian, Mughlai, Chinese",800.0,rated 40 ratedn beautiful place dine inthe int...,Banashankari,3.99


In [36]:
recommend_like('Jalsa')

TOP RESTAURANTS LIKE JALSA: 


,cuisines,Mean Rating,cost
0,"Continental, North Indian, Italian, South Indi...",5.00,1600.0
1,North Indian,4.96,2000.0
2,"Asian, Mediterranean, North Indian, BBQ",4.87,1400.0
3,BBQ,4.81,1300.0
4,"North Indian, European, Mediterranean, BBQ",4.78,1500.0
...,...,...,...
1571,"North Indian, Chinese, Continental",1.90,850.0
1572,South Indian,1.90,300.0
1573,Cafe,1.90,550.0
1574,"North Indian, Chinese, Fast Food, Rolls, Juices",1.90,400.0


# Recommendation by Cuisine

In [37]:
df_percent = data.sample(frac=1)
df_percent.set_index('cuisines'.lower(), inplace=True) 
indices = pd.Series(df_percent.index)

from fuzzywuzzy import fuzz

cuisine_mapping = {}
for index, row in data.iterrows():
    for cuisine in row['cuisines'].split(', '):
        normalized_cuisine = cuisine.lower()
        if normalized_cuisine not in cuisine_mapping:
            cuisine_mapping[normalized_cuisine] = []
        cuisine_mapping[normalized_cuisine].append(index)

def recommend_by_cuisine(criteria_value, top_n=-1, cosine_similarities=cosine_similarities):
    recommend_restaurant = []
    normalized_criteria = criteria_value.lower()
    matching_restaurants = cuisine_mapping.get(normalized_criteria, [])

    for restaurant_index in matching_restaurants:
        recommend_restaurant.append(restaurant_index)

    df_new = pd.DataFrame(columns=['name', 'cuisines', 'Mean Rating', 'cost'])
    for each in recommend_restaurant:
        selected_data = data[['name', 'cuisines', 'Mean Rating', 'cost']][data.index == each].sample()
        df_new = pd.concat([df_new, selected_data])
    df_new = df_new.drop_duplicates(subset=['name', 'cuisines', 'Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(top_n)

    print(f'TOP RESTAURANTS WITH SIMILAR CUISINE: ')
    df_new.reset_index(drop=True, inplace=True)
    return df_new.style.hide_index()

In [38]:
recommend_by_cuisine('North Indian')

TOP RESTAURANTS WITH SIMILAR CUISINE: 


name,cuisines,Mean Rating,cost
The Reservoire,"Continental, North Indian, Chinese, American, Pizza, Finger Food",4.560000,1300.000000
Levitate Brewery And Kitchen,"Finger Food, North Indian, Continental",4.480000,1500.000000
Output Bengaluru,"North Indian, Continental",4.480000,1000.000000
Feast - Sheraton Grand Bengaluru Whitefield Hotel &...,"Continental, Asian, South Indian, North Indian",4.480000,2500.000000
Saffron - Radisson Blu,"North Indian, Mughlai, Lucknowi",4.480000,2500.000000
Bitebox,"North Indian, Chinese, Fast Food, Beverages, Desserts",4.440000,300.000000
Indigo X Ii P,"Finger Food, Continental, Pizza, Asian, North Indian",4.350000,1250.000000
Salt - Indian Restaurant - Bar & Grill,North Indian,4.350000,1500.000000
The Creek - The Den Bengaluru,"North Indian, Continental, Chinese",4.350000,2000.000000
Sigree Global Grill,"Rajasthani, Gujarati, North Indian",4.350000,1400.000000


# Recommendation by Rating

In [39]:
df_percent = data.sample(frac=1)
df_percent.set_index('Mean Rating', inplace=True) 
indices = pd.Series(df_percent.index)

def recommend_by_mean_rating(mean_rating_value, top_n=-1):
    recommend_restaurant = data[data['Mean Rating'] >= mean_rating_value].index
    df_new = data.loc[recommend_restaurant, ['name', 'cuisines', 'Mean Rating', 'cost']]
    df_new = df_new.sample(frac=1) 
    df_new = df_new.head(top_n)
    print(f'TOP RESTAURANTS WITH A MINIMUM RATING OF {mean_rating_value}: ')
    df_new.reset_index(drop=True, inplace=True)
    return df_new

In [40]:
recommend_by_mean_rating(5)

TOP RESTAURANTS WITH A MINIMUM RATING OF 5: 


,name,cuisines,Mean Rating,cost
0,Asia Kitchen By Mainland China,"Asian, Chinese, Thai, Momos",5.0,1500.0
1,Byg Brewski Brewing Company,"Continental, North Indian, Italian, South Indi...",5.0,1600.0
2,Asia Kitchen By Mainland China,"Asian, Chinese, Thai, Momos",5.0,1500.0
3,Byg Brewski Brewing Company,"Continental, North Indian, Italian, South Indi...",5.0,1600.0
4,Asia Kitchen By Mainland China,"Asian, Chinese, Thai, Momos",5.0,1500.0
5,Asia Kitchen By Mainland China,"Asian, Chinese, Thai, Momos",5.0,1500.0
6,SantãÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃ...,"Healthy Food, Salad, Mediterranean",5.0,1000.0
7,Asia Kitchen By Mainland China,"Asian, Chinese, Thai, Momos",5.0,1500.0
8,Byg Brewski Brewing Company,"Continental, North Indian, Italian, South Indi...",5.0,1600.0
9,Asia Kitchen By Mainland China,"Asian, Chinese, Thai, Momos",5.0,1500.0


# Recommendation by Cost

In [41]:
df_percent = data.sample(frac=1)
df_percent.set_index('cost', inplace=True)  
indices = pd.Series(df_percent.index)

def recommend_by_cost(min_value,max_value, top_n=-1):
    recommend_restaurant = data[(data['cost'] > min_value) & (data['cost'] <= max_value)].index
    df_new = data.loc[recommend_restaurant, ['name', 'cuisines', 'Mean Rating', 'cost']]
    df_new = df_new.sample(frac=1) 
    df_new = df_new.head(top_n)
    print(f'TOP RESTAURANTS IN THE PROVIDED RANGE: ')
    df_new.reset_index(drop=True, inplace=True)
    return df_new

In [43]:
recommend_by_cost(1000,2000)

TOP RESTAURANTS IN THE PROVIDED RANGE: 


,name,cuisines,Mean Rating,cost
0,Crawl Street,"Continental, Finger Food, North Indian, Chinese",4.22,1200.0
1,Smaaash,"North Indian, Pizza, Chinese",4.06,1500.0
2,Fresh Pressery Cafe,"Cafe, Italian, Healthy Food",3.95,1200.0
3,Easy Tiger,"Finger Food, Continental",4.23,1100.0
4,Chutney Chang,"North Indian, Chinese, BBQ",4.05,1500.0
...,...,...,...,...
4506,Farzi Cafe,Modern Indian,4.35,1500.0
4507,Izakaya Gastro Pub,"North Indian, Continental, Mediterranean",3.58,1200.0
4508,Murphy'S Brewhouse - The Paul Bangalore,"Continental, North Indian, Finger Food",4.23,1700.0
4509,Sidewalk Cafe & Bar - The Paul Bangalore,"European, Italian, Asian",4.10,1500.0
